In [47]:
# 내가 필요한 함수와 라이브러리를 import한다.
from bs4 import BeautifulSoup
from requests import get
from re import compile, findall, sub

# Just a function that turns lists into strings because the re.findall() really annoys me
def lststr(lst):
    string = ''
    return string.join(lst)

# 이 .py 파일을 실행하면 실행될 main method이다.
def main():
    # 혐오 표현을 "게시물 검색"으로 서치하면 나오는 결과 페이지 링크이다.
    link = input("DCINSIDE 혐오표현 검색 페이지 링크: ")
    #link = "https://search.dcinside.com/post/q/.ED.96.89.EB.B3.B5"
    
    """디젤매니아 예시(검색 키워드 : 페미) :
    https://cafe.daum.net/_c21_/cafesearch?grpid=10akR&item=subject&sorttype=0&query=%ED%8E%98%EB%AF%B8#"""

    #그 페이지의 html을 text로 가져온다.
    html_text = get(link).text

    # 그 html 내용을 좀 더 다루기 쉽게 BeautifulSoup로 만든다.
    soup = BeautifulSoup(html_text, 'lxml')
    
    """ 디씨의 경우(기존 코드):

    # 원하는 url들이 있는 class를 찾는다.
    sch_list= soup.find_all('a', class_= 'tit_txt')

    디씨인사이드의 경우 'tit.txt' 태그가 먹히지만 다른 곳은 먹히지 않는다.
    
    디젤매니아의 경우 html 코드를 뜯어보면 <tr class='list_row_info'> 안에서 다시 한 번 td 태그로 들어가고, 거기서 a 태그를 추출한 뒤
    이후는 디씨에서 했던 것과 동일하게 정규식으로 href 태그에서 큰따옴표 안의 링크를 추출한다.
    구체적인 구조는 div id 'container' -> div id 'content' -> div class 'primary content' -> table 'class roundTable'
    -> tbody class 'pos_rel' -> form id 'innerSearchForm' -> table id 'searchCafeList' -> tr class 'list_row_info'다.
    
    BeautifulSoup에서 find로 td 태그의 id와 class 조건을 다는 것은 불가능하기 때문에 (soup.find('td', class_ = '클래스명') 불가능)
    리스트의 index를 이용하여 원하는 부분을 추출한다.
    """

    """다수의 tr 태그에 검색 결과가 저장되어 있기 때문에 findAll을 이용해 전체 검색 결과를 리스트에 담는다."""
    div_list = soup.findAll('tr', class_ = 'list_row_info')
    a_list=[]
    
    for link in div_list:
        tmp_list = link.findAll('td')
        str_tmp = tmp_list[1].find('a')
        a_list.append(str_tmp)
        
    """a_list는 기존 코드의 sch_list와 동일하다. 해당 리스트에 a 태그를 저장한다."""

    # 그 안에서 링크만 쏙 빼오고 싶기 때문에 regular expression(정규 표현)을 사용한다.
    r = compile('href="(.*?)"')
    # for loop을 위해 비워둔 리스트
    url_list = []

    for link in a_list:
        
        # re library는 string을 input으로 받아야하기 때문에 str()을 사용한다.
        link = str(link)
        # re library를 사용해 링크 부분만 빼온다. 그러나 findall()은 list를 return한다. ㅂㄷㅂㄷ!!!
        url = r.findall(link)
        # 그래서 아까 만들어둔 함수를 사용한다 ㅎ
        url_str = lststr(url)
        #비워둔 리스트에 추가해준다.
        url_list.append(url_str)

    #다른 사이트에서도 이런 문제가 발생하는 지는 모르겠지만, "&"가 "&amp;"로 바뀌어 가져와지는 현상이 일어났다.
    #그것도 고쳐주자. 안 그러면 링크가 먹통이다.
    new_list = []
    #링크에서 에러나는 부분들을 고쳐준다.
    for url in url_list:
        """디젤 매니아에서 추출한 <a href>의 경우 앞부분에 따로 주소를 붙여주어야 링크가 기능한다.
        참고로 카페에 가입해야 검색 결과를 볼 수 있기 때문에 가입 후 내용 체크가 가능하다."""
        url = "https://cafe.daum.net/"+url
        new_list.append(sub('&amp;', "&", url))

    # 제대로 모든 게 됐는 지 확인하는 for loop.
    for url in new_list:
        print(url)

    #url들을 txt파일에 저장한다.
    with open("url_list.txt", "w") as f:
        for url in new_list:
            f.write(url + "\n")
#이 파링을 실행하면 main method를 실행하도록 한다.
if __name__ == "__main__":
    main()
    
    
"""기본적으로 모든 웹페이지의 구조는 통일 되어 있지 않기 때문에 '보편적인 링크 추출기'는 힘들고, 
    찾아보고자 하는 페이지에 맞게 수정하는 과정이 필요하다.
    
    또한 현재 코드의 경우 1페이지의 검색 결과만을 보여주기 때문에(디씨인사이드, 디젤매니아 둘 다 포함) 다른 페이지의 결과를 찾고 싶다면
    for문을 이용하여 추출 과정을 반복하면 된다. 이때 디씨인사이드의 경우 페이지 주소의 중간 부분에  'p/1/', 'p/2/'등의 값을 넣어주어야
    하며, 아래 코드는 해당 반복 과정을 반영한 것.(디젤매니아는 검색 결과가 1페이지를 넘지 않아서 디씨인사이드로 지정.)
"""

DCINSIDE 혐오표현 검색 페이지 링크: https://cafe.daum.net/_c21_/cafesearch?grpid=10akR&item=subject&sorttype=0&query=%ED%8E%98%EB%AF%B8#
https://cafe.daum.net//_c21_/bbs_nsread?grpid=10akR&fldid=JTgk&contentval=01VhBzzzzzzzzzzzzzzzzzzzzzzzzz&datanum=360169&searchlist_uri=%2F_c21_%2Fcafesearch&search_ctx=LrpNuZSucOsx.YncUoV2ej_tRpHqkOL1hFNwnK7pMwx2pUU9oYvq-CADq-lzJK_p_TYbLZznvSbg3qy8mowcMHPywQhEukJ9BUhf51WKvlb6vqePL4uMQqcs48Wyoau45p9vy72sItOpaHn.XqjkIM-e6rMKz8l9jqqrMAoL_H.Tf4YXEx2CtXqiraQtm3YAI3XDX3WyUrJduDtnsIwt_xoTaIvM9IeQ_z7YootSc29RNaqzIwmc954q1Gdyb6DDSkZkZj1SVmOJeAOTl6L4_a1K3m3huQ-mtc8_CHa8m.DuCN82Qv4iK.ImWdsjTez.
https://cafe.daum.net//_c21_/bbs_nsread?grpid=10akR&fldid=JTgk&contentval=01UlGzzzzzzzzzzzzzzzzzzzzzzzzz&datanum=356578&searchlist_uri=%2F_c21_%2Fcafesearch&search_ctx=LrpNuZSucOsx.YncUoV2ej_tRpHqkOL1hFNwnK7pMwx2pUU9oYvq-CADq-lzJK_p_TYbLZznvSbg3qy8mowcMHPywQhEukJ9BUhf51WKvlb6vqePL4uMQqcs48Wyoau45p9vy72sItOpaHn.XqjkIM-e6rMKz8l9jqqrMAoL_H.Tf4YXEx2CtXqiraQtm3YAI3XDX3WyUrJduDtnsIwt_xoTa

"기본적으로 모든 웹페이지의 구조는 통일 되어 있지 않기 때문에 '보편적인 링크 추출기'는 힘들고, \n    찾아보고자 하는 페이지에 맞게 수정하는 과정이 필요하다.\n    \n    또한 현재 코드의 경우 1페이지의 검색 결과만을 보여주기 때문에(디씨인사이드, 디젤매니아 둘 다 포함) 다른 페이지의 결과를 찾고 싶다면\n    for문을 이용하여 추출 과정을 반복하면 된다. 이때 디씨인사이드의 경우 페이지 주소의 중간 부분에  'p/1/', 'p/2/'등의 값을 넣어주어야\n    하며, 아래 코드는 해당 반복 과정을 반영한 것.(디젤매니아는 검색 결과가 1페이지를 넘지 않아서 디씨인사이드로 지정.)\n"

In [50]:
# 내가 필요한 함수와 라이브러리를 import한다.
from bs4 import BeautifulSoup
from requests import get
from re import compile, findall, sub

# Just a function that turns lists into strings because the re.findall() really annoys me
def lststr(lst):
    string = ''
    return string.join(lst)

# 이 .py 파일을 실행하면 실행될 main method이다.
def main():
    # 혐오 표현을 "게시물 검색"으로 서치하면 나오는 결과 페이지 링크이다.
    link = input("DCINSIDE 혐오표현 검색 페이지 링크: ")
    tmp_link = link
    # link = "https://search.dcinside.com/post/q/.ED.96.89.EB.B3.B5"
    """페이지가 지정된 링크 :
        https://search.dcinside.com/post/p/1/q/.ED.96.89.EB.B3.B5
    """
    
    """페이지 수가 끝인지 아닌지 비교하기 위한 임시 string 변수"""
    tmp_str = ''
    
    """임시로 최대 200페이지까지 잡았지만 더 늘려도 OK. 아래 tmp_link는 입력한 링크 중간에 페이지 수를 끼워넣음"""
    for i in range(1, 200):
        tmp_link = link[:32]+"/p/"+str(i)+"/"+link[33:]

    #그 페이지의 html을 text로 가져온다.
        html_text = get(tmp_link).text

    # 그 html 내용을 좀 더 다루기 쉽게 BeautifulSoup로 만든다.
        soup = BeautifulSoup(html_text, 'lxml')
    # 원하는 url들이 있는 class를 찾는다.
        sch_list= soup.find_all('a', class_= 'tit_txt')
    # 그 안에서 링크만 쏙 빼오고 싶기 때문에 regular expression(정규 표현)을 사용한다.
        r = compile('href="(.*?)"')
    # for loop을 위해 비워둔 리스트
        url_list = []

        for link2 in sch_list:
        # re library는 string을 input으로 받아야하기 때문에 str()을 사용한다.
            link2 = str(link2)
        # re library를 사용해 링크 부분만 빼온다. 그러나 findall()은 list를 return한다. ㅂㄷㅂㄷ!!!
            url = r.findall(link2)
        # 그래서 아까 만들어둔 함수를 사용한다 ㅎ
            url_str = lststr(url)
        #비워둔 리스트에 추가해준다.
            url_list.append(url_str)

    #다른 사이트에서도 이런 문제가 발생하는 지는 모르겠지만, "&"가 "&amp;"로 바뀌어 가져와지는 현상이 일어났다.
    #그것도 고쳐주자. 안 그러면 링크가 먹통이다.
        new_list = []
    #링크에서 에러나는 부분들을 고쳐준다.
        for url in url_list:
            new_list.append(sub('&amp;', "&", url))
        
        """이전 페이지의 첫 번째 주소와 다음 페이지의 첫번째 주소가 동일하면 더 이상 페이지가 없는 것으로 간주하고 for문을 빠져나옴"""
        if(tmp_str == new_list[0]):
            break
        else:    
            tmp_str = new_list[0]

    # 제대로 모든 게 됐는 지 확인하는 for loop.
        for url in new_list:
            print(url)

    #url들을 txt파일에 저장한다.
        with open("url_list.txt", "w") as f:
            for url in new_list:
                f.write(url + "\n")
            
#이 파링을 실행하면 main method를 실행하도록 한다.
if __name__ == "__main__":
    main()

DCINSIDE 혐오표현 검색 페이지 링크: https://search.dcinside.com/post/q/.ED.96.89.EB.B3.B5
https://search.dcinside.com/post/p/1/q/.ED.96.89.EB.B3.B5
https://gall.dcinside.com/board/view/?id=spman&no=546143
https://gall.dcinside.com/board/view/?id=gf&no=3951825
https://gall.dcinside.com/mgallery/board/view/?id=bser&no=1946568
https://gall.dcinside.com/board/view/?id=hanwhaeagles_new&no=5075937
https://gall.dcinside.com/board/view/?id=leagueoflegends4&no=6335210
https://gall.dcinside.com/board/view/?id=sbsdocu&no=870459
https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=1633774
https://gall.dcinside.com/mgallery/board/view/?id=youth_go&no=37500
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8420213
https://gall.dcinside.com/board/view/?id=coronavirus&no=720929
https://gall.dcinside.com/mgallery/board/view/?id=genrenovel&no=3762751
https://gall.dcinside.com/mgallery/board/view/?id=prospect&no=38970
https://gall.dcinside.com/mgallery/board/view/?id=billboard&no=96672

https://gall.dcinside.com/mgallery/board/view/?id=espressocookie&no=529604
https://gall.dcinside.com/board/view/?id=gf&no=3951690
https://gall.dcinside.com/board/view/?id=sbsdocu&no=870449
https://gall.dcinside.com/mgallery/board/view/?id=kizunaai&no=2895492
https://gall.dcinside.com/mgallery/board/view/?id=entp&no=250466
https://gall.dcinside.com/board/view/?id=fifaonline&no=3150715
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8418861
https://gall.dcinside.com/board/view/?id=streetwomanfighter&no=271298
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8418839
https://gall.dcinside.com/board/view/?id=d_fighter_new2&no=1028746
https://gall.dcinside.com/mini/board/view/?id=snp500&no=174567
https://gall.dcinside.com/mgallery/board/view/?id=dnfqq&no=3413964
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8418820
https://gall.dcinside.com/mgallery/board/view/?id=bang_dream&no=3622223
https://gall.dcinside.com/mgallery/board/view/?id=dsrqsv&no=254142
https:

https://gall.dcinside.com/board/view/?id=divination_new1&no=251114
https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=1633025
https://gall.dcinside.com/board/view/?id=giants_new2&no=8128171
https://gall.dcinside.com/mgallery/board/view/?id=arcaea&no=127694
https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=1633014
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8417733
https://gall.dcinside.com/board/view/?id=grsgills&no=710450
https://gall.dcinside.com/mgallery/board/view/?id=game_nintendo&no=1245915
https://gall.dcinside.com/mgallery/board/view/?id=indiegame&no=16275
https://gall.dcinside.com/mgallery/board/view/?id=entp&no=250419
https://gall.dcinside.com/mgallery/board/view/?id=ssawagent&no=331
https://gall.dcinside.com/mgallery/board/view/?id=gfl2&no=5597683
https://gall.dcinside.com/board/view/?id=squid456&no=84321
https://gall.dcinside.com/mgallery/board/view/?id=derrickfavors&no=111632
https://gall.dcinside.com/mgallery/b

https://gall.dcinside.com/board/view/?id=baseball_new10&no=8416719
https://gall.dcinside.com/mgallery/board/view/?id=hjp&no=35170
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8416713
https://gall.dcinside.com/board/view/?id=bitcoins_new1&no=61877
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8416701
https://gall.dcinside.com/mgallery/board/view/?id=aimotorinku&no=74
https://gall.dcinside.com/mgallery/board/view/?id=jhkim&no=19020
https://gall.dcinside.com/mgallery/board/view/?id=umamusme&no=730598
https://gall.dcinside.com/board/view/?id=lostark&no=7298048
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8416666
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8416660
https://gall.dcinside.com/mgallery/board/view/?id=mfgo&no=3411720
https://gall.dcinside.com/mgallery/board/view/?id=worldflipper&no=469636
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8416633
https://gall.dcinside.com/board/view/?id=baseball_new10&no=8416624
htt

https://gall.dcinside.com/mgallery/board/view/?id=buddhism&no=31768
https://gall.dcinside.com/board/view/?id=divination_new1&no=250214
https://gall.dcinside.com/mgallery/board/view/?id=hoooonoka0128&no=3795
https://gall.dcinside.com/board/view/?id=fantasy_new2&no=240744
https://gall.dcinside.com/mgallery/board/view/?id=vr&no=1298230
https://gall.dcinside.com/mgallery/board/view/?id=leechanwon&no=149637
https://gall.dcinside.com/mgallery/board/view/?id=otoko&no=613440
https://gall.dcinside.com/mgallery/board/view/?id=jooheon&no=1413
https://gall.dcinside.com/mgallery/board/view/?id=tg74&no=25288
https://gall.dcinside.com/mgallery/board/view/?id=onairpril&no=231216
https://gall.dcinside.com/mgallery/board/view/?id=leechanwon&no=149572
https://gall.dcinside.com/mgallery/board/view/?id=genrenovel&no=3761255
https://gall.dcinside.com/board/view/?id=lostark&no=7296364
https://gall.dcinside.com/mgallery/board/view/?id=kirafan&no=76279
https://gall.dcinside.com/board/view/?id=d_fighter_new2&no

...

https://gall.dcinside.com/board/view/?id=fantasy_new2&no=235723
https://gall.dcinside.com/mgallery/board/view/?id=guardiantales&no=3290665
https://gall.dcinside.com/board/view/?id=baseball_ab2&no=2082014
https://gall.dcinside.com/mgallery/board/view/?id=purikone_redive&no=6500046
https://gall.dcinside.com/board/view/?id=kart&no=2866860
https://gall.dcinside.com/board/view/?id=boardgame&no=3067313
https://gall.dcinside.com/mini/board/view/?id=ragorigin&no=3862
https://gall.dcinside.com/mgallery/board/view/?id=shinycolors&no=794861
https://gall.dcinside.com/board/view/?id=baseball_ab2&no=2081613
https://gall.dcinside.com/mgallery/board/view/?id=fm&no=14754
https://gall.dcinside.com/mgallery/board/view/?id=sosad&no=75532
https://gall.dcinside.com/mgallery/board/view/?id=diablo2resurrected&no=267821
https://gall.dcinside.com/mgallery/board/view/?id=jtbclost&no=1104
https://gall.dcinside.com/mgallery/board/view/?id=namuwiki&no=32219
https://gall.dcinside.com/board/view/?id=wwe&no=2103808
ht

https://gall.dcinside.com/board/view/?id=fantasy_new2&no=235300
https://gall.dcinside.com/mgallery/board/view/?id=twitter&no=945423
https://gall.dcinside.com/mgallery/board/view/?id=physics2&no=58462
https://gall.dcinside.com/mgallery/board/view/?id=idolsy&no=155589
https://search.dcinside.com/post/p/101/q/.ED.96.89.EB.B3.B5
https://gall.dcinside.com/board/view/?id=d_fighter_new2&no=1022056
https://gall.dcinside.com/mgallery/board/view/?id=penpal&no=109816
https://gall.dcinside.com/mgallery/board/view/?id=smwinter&no=3372
https://gall.dcinside.com/board/view/?id=nba&no=6427593
https://gall.dcinside.com/board/view/?id=football_new7&no=2221529
https://gall.dcinside.com/board/view/?id=infinite&no=3060571
https://gall.dcinside.com/mgallery/board/view/?id=onshinproject&no=3993329
https://gall.dcinside.com/mgallery/board/view/?id=juhwanno&no=20496
https://gall.dcinside.com/mgallery/board/view/?id=billboard&no=964047
https://gall.dcinside.com/board/view/?id=immovables&no=4052476
https://gall.

https://gall.dcinside.com/mgallery/board/view/?id=qmfzh&no=675179
https://gall.dcinside.com/board/view/?id=mabi_heroes2&no=576158
https://gall.dcinside.com/mgallery/board/view/?id=buddhism&no=31740
https://gall.dcinside.com/board/view/?id=getcha&no=24740
https://gall.dcinside.com/board/view/?id=skwyverns_new1&no=133373
https://gall.dcinside.com/mgallery/board/view/?id=btsv&no=276869
https://gall.dcinside.com/mgallery/board/view/?id=tenbagger&no=3982544
https://gall.dcinside.com/mini/board/view/?id=penthousedsb&no=7086
https://gall.dcinside.com/board/view/?id=correction&no=446544
https://gall.dcinside.com/mgallery/board/view/?id=higurashi&no=279179
https://gall.dcinside.com/board/view/?id=iu_new&no=6204590
https://gall.dcinside.com/mgallery/board/view/?id=2nara&no=135057
https://gall.dcinside.com/mgallery/board/view/?id=kizunaai&no=2880810
https://gall.dcinside.com/board/view/?id=maplestory&no=16150579
https://gall.dcinside.com/board/view/?id=joy&no=537332
https://gall.dcinside.com/mgal

https://gall.dcinside.com/mgallery/board/view/?id=bser&no=1940934
https://gall.dcinside.com/board/view/?id=divination_new1&no=242768
https://gall.dcinside.com/board/view/?id=idolmaster_new1&no=261217
https://gall.dcinside.com/mgallery/board/view/?id=diablo2resurrected&no=262156
https://gall.dcinside.com/board/view/?id=lgbt&no=4491422
https://gall.dcinside.com/board/view/?id=divination_new1&no=242757
https://gall.dcinside.com/board/view/?id=maplestory&no=16150116
https://gall.dcinside.com/mgallery/board/view/?id=onshinproject&no=3991469
https://gall.dcinside.com/board/view/?id=squid456&no=79453
https://gall.dcinside.com/mgallery/board/view/?id=kiryong2&no=50525
https://gall.dcinside.com/board/view/?id=divination_new1&no=242740
https://gall.dcinside.com/board/view/?id=m_entertainer_new1&no=342546
https://gall.dcinside.com/board/view/?id=brahms&no=93533
https://gall.dcinside.com/mgallery/board/view/?id=onshinproject&no=3991427
https://gall.dcinside.com/mgallery/board/view/?id=shinycolors&

https://gall.dcinside.com/mgallery/board/view/?id=umamusme&no=729010
https://gall.dcinside.com/mgallery/board/view/?id=stockus&no=1945334
https://gall.dcinside.com/mgallery/board/view/?id=euca&no=1008076
https://gall.dcinside.com/mgallery/board/view/?id=vr&no=1293577
https://gall.dcinside.com/mgallery/board/view/?id=mnet_k&no=19964581
https://gall.dcinside.com/mgallery/board/view/?id=blnovel&no=3565730
https://gall.dcinside.com/mgallery/board/view/?id=iz4648&no=156846
https://gall.dcinside.com/mgallery/board/view/?id=lovecome&no=168707
https://gall.dcinside.com/mgallery/board/view/?id=jungsewoon&no=610441
https://gall.dcinside.com/board/view/?id=game_classic1&no=4084259
https://gall.dcinside.com/board/view/?id=lostark&no=7273720
https://gall.dcinside.com/mgallery/board/view/?id=planetguardian&no=71096
https://gall.dcinside.com/mgallery/board/view/?id=kizunaai&no=2879305
https://gall.dcinside.com/board/view/?id=kdani&no=2228998
https://gall.dcinside.com/mgallery/board/view/?id=dontgo28&